In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output

In [ ]:
s = 20

In [ ]:
class Grass:
    def __init__(self, world, x, y, poison=False):
        self.x, self.y = x, y
        self.world = world
        self.id = None

    def die(self):
        self.world.field[self.y, self.x] = 0  
        del self.world.objects[self.id]
    
    def tick(self):
        pass

class Animal:
    def __init__(self, world, x, y, predator=False):
        self.x, self.y = x, y
        self.view_dist = 10
        self.id = None
        self.hungry = 0
        self.world = world
        self.predator = predator

    def look_for_food_nearby(self):
        food = []
        food_types = [Animal] if self.predator else [Grass]
        i, j = self.y, self.x
        grid = self.world.field
        for k in range(i-1, i+2):
            for p in range(j-1, j+2):
                if k == i and p == j:
                    continue
                if not self.predator:
                    if abs(k-i) + abs(p-j) == 2:
                        continue
                if k >= len(grid):
                    k = k % len(grid)
                if p >= len(grid[0]):
                    p = p % len(grid[0])

                if grid[k][p] != 0 and type(self.world.objects[grid[k][p]]) in food_types:
                    food.append(self.world.objects[grid[k][p]])
        return food
    
    def look_for_food_far(self):
        food = []
        food_types = [Animal] if self.predator else [Grass]
        i, j = self.y, self.x
        grid = self.world.field
        for k in range(i-self.view_dist, i+self.view_dist+1):
            for p in range(j-self.view_dist, j+self.view_dist+1):
                if k == i and p == j:
                    continue
                if k >= len(grid):
                    k = k % len(grid)
                if p >= len(grid[0]):
                    p = p % len(grid[0])

                if grid[k][p] != 0 and type(self.world.objects[grid[k][p]]) in food_types:
                    food.append((self.world.objects[grid[k][p]], np.sqrt((k-i)**2 + (p-j)**2)))
        food =  sorted(food, key=lambda x: -x[-1])
        return food
        
    def tick(self):
        self.hungry += 1
        if self.hungry > 100:
            self.die()
            return
        
        food = self.look_for_food_nearby()

        if len(food) > 0:
            self.eat(food[0])
            return
        else:
            food = self.look_for_food_far()

        if len(food) > 0:
            f = food[0][0]
            dir_y = f.y - self.y
            dir_x = f.x - self.x
            dir_y = int(dir_y / max(abs(dir_y), 1))
            dir_x = int(dir_x / max(abs(dir_x), 1))
        else:
            dir_x, dir_y = np.random.randint(-1, 2, 2)

        if not self.predator and (abs(dir_y) + abs(dir_x) == 2):
            b = np.random.randint(0, 2, dtype=int)
            dir_y *= b
            dir_x *= (1-b)
        self.move((self.x + dir_x) % len(self.world.field[0]), 
                  (self.y + dir_y) % len(self.world.field))
            
    def move(self, x, y):
        self.world.field[self.y, self.x] = 0
        self.x, self.y = x, y
        self.world.field[self.y, self.x] = self.id

    def die(self):
        self.world.field[self.y, self.x] = 0  
        del self.world.objects[self.id]
        
    def eat(self, obj):
        if type(obj) == Animal:
            if self.predator:
                self.hungry -= 20
                x, y = obj.x, obj.y
                obj.die()
                self.world.field[self.y, self.x] = 0
                self.x, self.y = x, y
                self.world.field[self.y, self.x] = self.id
        else:
            if not self.predator:
                self.hungry -= 5
                x, y = obj.x, obj.y
                obj.die()
                self.world.field[self.y, self.x] = 0
                self.x, self.y = x, y
                self.world.field[self.y, self.x] = self.id
            

class World:
    def __init__(self, size_x, size_y):
        self.field = np.zeros((size_y, size_x))
        self.objects = {}
        self.id_c = 0

    def add_obj(self, obj):
        obj.id = self.id_c
        self.id_c += 1
        if self.field[obj.y, obj.x] == 0:
            self.field[obj.y, obj.x] = obj.id
            self.objects[obj.id] = obj
    
    def tick(self):
        ids = list(self.objects.keys())
        global i
        if np.random.randint(0, 2):
            for k in range(10):
                self.add_obj(Grass(self, np.random.randint(0, s), np.random.randint(0, s)))
                i += 1
        
        if np.random.randint(0, 2):
            for k in range(10):
                self.add_obj(Animal(self, np.random.randint(0, s), np.random.randint(0, s), np.random.randint(0, 2)))
                i += 1

        for idx in ids:
            if idx in self.objects.keys():
                self.objects[idx].tick()

In [ ]:

w = World(s, s)

In [ ]:
i = 0

In [ ]:
for k in range(100):
    w.add_obj(Grass(w, np.random.randint(0, s), np.random.randint(0, s)))
    i += 1
    

In [ ]:
for k in range(30):
    w.add_obj(Animal(w, np.random.randint(0, s), np.random.randint(0, s), np.random.randint(0, 2)))
    i += 1

In [ ]:
for i in range(1000):
    xs, ys = {Animal:{True:[], False:[]}, Grass:[]}, {Animal:{True:[], False:[]}, Grass:[]}
    for obj in w.objects.values():
        if type(obj) == Animal:
            xs[type(obj)][obj.predator].append(obj.x)
            ys[type(obj)][obj.predator].append(obj.y)
        else:
            xs[type(obj)].append(obj.x)
            ys[type(obj)].append(obj.y)

    clear_output(True)
    plt.figure()
    plt.scatter(xs[Grass], ys[Grass], c='g')
    plt.scatter(xs[Animal][True], ys[Animal][True], c='r')
    plt.scatter(xs[Animal][False], ys[Animal][False], c='b')
    plt.show()
    w.tick()

KeyboardInterrupt: ignored

<Figure size 432x288 with 0 Axes>